In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

import nltk
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [2]:
file1 = 'data/QueryResults3_new.csv'
df1 = pd.read_csv(file1,index_col=['Id'])

import mes_fonctions_final2 as stt

In [3]:
StopWord = stt.load_stop_word()

## Chargement des Tags

In [4]:
Y_tags, Y_tags_voc = stt.genere_target_dummy_and_vocabulary(data=df1, min_df=300)
Tags_freq = stt.genere_df_target_tags_freq(data=df1, min_df=300)

/!\ Ajout des features en cours


# Fonctions de pré-processing

In [5]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()

def preProcess_remove_Code(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    # Suppression des blocs de Code
    for h_code in html_in_soup.find_all('code'):
        h_code.decompose()

    return html_in_soup.get_text().lower()


def preProcess_keep_Code(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    return html_in_soup.get_text().lower()

# Le préprocessing :
#   - nettoie le html
#   - supprime les blocs de Code
#   - ne prend que la partie textuelle du html
#   - renvoie ce texte en minuscule


def preProcess(html_in):
    # Chargement du module BeautifulSoup pour le parsing des données HTML
    from bs4 import BeautifulSoup
    html_in_soup = BeautifulSoup(html_in, 'html.parser')

    # Suppression des blocs de Code
    for h_code in html_in_soup.find_all('code'):
        h_code.decompose()

    return html_in_soup.get_text().lower()


# Réalisation du stemming (on coupe les racines)
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


# La tokenisation :


def tokenize(text):
    m_token_pattern = r"((?:(?:(?:[0-9a-zA-Z])\.){2,}[a-zA-Z])" +\
        "|(?:(?:[0-9a-zA-Z]){2,}\.(?:[0-9a-zA-Z]){2,}" +\
        "|(?:\.(?:[0-9a-zA-Z]){2,}))" +\
        "|[0-9a-zA-Z-\-\+\#]{2,}|w+)"

    from nltk.tokenize import RegexpTokenizer
    # Nous allons utiliser le pattern pour identifier les mots
    tokenizer = RegexpTokenizer(m_token_pattern)

    # Nous lançons la séparation des mots
    tokens = tokenizer.tokenize(text)

    # On fait appel au stemming pour rapprocher les mots de même racine
    stems = stem_tokens(tokens, stemmer)

    # Etape de nettoyage des valeurs :
    # Nous allons supprimer les nombres sans texte,
    def suppress_nb(x):
        import re
        if x is None:
            return None
        pattern = r'(^[\#\-\+]*[0-9]*$|' + '^[0-9]*[\#\-\+]*$|'+'^[0-9]*[\#\-\+]?[0-9]*$|'+'^[0-9\#\-\+][a-z]$|'+'^[a-z][0-9\#\-\+]$|'+'^[0-9]*\.[0-9]*$)'
        if not(re.match(pattern, x)):
            return x

    def nettoie_points(x):
        import re
        if x is None:
            return None

        if (re.match(r'(^[\.\-\#][a-z]*$)', x)):
            return ''.join(list(x)[1:])
        else:
            return x

    # Nous filtrons les nombres seuls
    stems = list(filter(lambda x: suppress_nb(x), stems))
    stems = [nettoie_points(x) for x in stems]
    return stems

## Filtre de X_train 

### Ajout de Body_clean_token
> On nettoie les Body en appliquant le même nettoyage que celui réalisé pour la version des BOW traditionnelle

In [6]:
%%time 

df1['Body_clean_token'] = df1.Body.apply(lambda x: ' '.join(tokenize(x)))

CPU times: user 1min 54s, sys: 191 ms, total: 1min 55s
Wall time: 1min 55s


In [7]:
# On sépare les données d'entrainement de celles de tests
debut = datetime.datetime.now()
X = df1['Body']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_tags, test_size=0.2, random_state=42 )
fin = datetime.datetime.now()
print("Fin Train_Split : {}".format(fin-debut))

Fin Train_Split : 0:00:00.069478


## Séparation Title / Body

In [8]:
# VERSION TITLE
X_train_Title = df1.loc[X_train.index, 'Title']
X_test_Title = df1.loc[X_test.index, 'Title']

train_title = pd.DataFrame(X_train_Title)
test_title = pd.DataFrame(X_test_Title)
train_title.columns = ['post']
test_title.columns = ['post']

# VERSION BODY
###X_train_body = df1.loc[X_train.index, 'Body']
###X_test_body = df1.loc[X_test.index, 'Body']
X_train_body = df1.loc[X_train.index, 'Body_clean_token']
X_test_body = df1.loc[X_test.index, 'Body_clean_token']


train_body = pd.DataFrame(X_train_body)
test_body = pd.DataFrame(X_test_body)
train_body.columns = ['post']
test_body.columns = ['post']

## ELMO 

#### Librairies

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


#### Chargement du modèle pré-entrainé d'ELMO

In [10]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

INFO:tensorflow:Using /var/folders/95/fydcf18x5rz7wljn4wqt_tq00000gn/T/tfhub_modules to cache modules.


##### Transformation des données Cibles par un encodage 
> On ne traite que les tags Python

In [11]:
## Y 
y1 = np.where(Y_train['python'].to_dense()==0, 'non', 'python')
y2 = np.where(Y_test['python'].to_dense()==0, 'non', 'python')

le = preprocessing.LabelEncoder()
le.fit(y1)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_train = np.asarray(encode(le, y1))
y_test = np.asarray(encode(le, y2))

In [12]:
## On transforme les features en np.array : 
x_train_title = np.asarray(X_train_Title)
x_train_body = np.asarray(X_train_body)

x_test_title = np.asarray(X_test_Title)
x_test_body = np.asarray(X_test_body)

##### Configuration de la Partie Title
> On paramètre le plongement de mots pour le titre

In [13]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
    #return embed(tf.squeeze(tf.cast(x, tf.string)), signature="tokens", as_dict=True)["tokens"]

input_text_title = Input(shape=(1,), dtype=tf.string)
embedding_title = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text_title)
dense_title = Dense(256, activation='relu')(embedding_title)
pred_title = Dense(2, activation='softmax')(dense_title)
model_title = Model(inputs=[input_text_title], outputs=pred_title)
model_title.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


##### Partie Body
> On paramètre le plongement de mots pour le body

In [14]:
input_text_body = Input(shape=(1,), dtype=tf.string)
embedding_body = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text_body)
dense_body = Dense(256, activation='relu')(embedding_body)
pred_body = Dense(2, activation='softmax')(dense_body)
model_body = Model(inputs=[input_text_body], outputs=pred_body)
model_body.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


> On paramètre le plongement de mots pour le body nettoyé

In [15]:
input_text_body = Input(shape=(1,), dtype=tf.string)
embedding_body = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text_body)
dense_body = Dense(256, activation='relu')(embedding_body)
pred_body = Dense(2, activation='softmax')(dense_body)
model_body_V2 = Model(inputs=[input_text_body], outputs=pred_body)
model_body_V2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


# Entrainement

##### BODY CLEAN TOKEN

In [16]:
# VERSION BODY V2
X_train_body_V2 = df1.loc[X_train.index, 'Body_clean_token']
X_test_body_V2 = df1.loc[X_test.index, 'Body_clean_token']

train_body_V2 = pd.DataFrame(X_train_body_V2)
test_body_V2 = pd.DataFrame(X_test_body_V2)
train_body_V2.columns = ['post']
test_body_V2.columns = ['post']

## X V2: 
x_train_body_V2 = np.asarray(X_train_body_V2)
x_test_body_V2 = np.asarray(X_test_body_V2)

### Entrainement du RNN pour le Body nettoyé

In [17]:
%%time 

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history_body_V2 = model_body_V2.fit(x_train_body_V2, y_train, epochs=1, batch_size=8)
    model_body_V2.save_weights('./MODELS/elmo-model-body_clean_token.h5')

Epoch 1/1
40000/40000 [==============================] - 64621s 2s/step - loss: 0.2920 - acc: 0.8949
CPU times: user 4d 16h 57min 11s, sys: 3h 29min 41s, total: 4d 20h 26min 52s
Wall time: 17h 57min 2s


> L'Entrainement aura duré 18h 

### Entrainement du RNN pour le Titre

In [18]:
%%time 
#NB_VALUES = 10

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    #####history_title = model_title.fit(x_train_title, y_train, epochs=1, batch_size=32)
    history_title = model_title.fit(x_train_title, y_train, epochs=1, batch_size=8)
    model_title.save_weights('./MODELS/elmo-model-title.h5')

Epoch 1/1
40000/40000 [==============================] - 2171s 54ms/step - loss: 0.2742 - acc: 0.9124
CPU times: user 3h 40min 2s, sys: 7min 50s, total: 3h 47min 53s
Wall time: 36min 12s


> L'Entrainement est ici beaucoup plus rapide : 36 min car il y a beaucoup moins de mots

# Prédiction & Test

#### Body Clean Token

In [19]:
%%time 

### VERSION BODY CLEAN
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model_body_V2.load_weights('./MODELS/elmo-model-body_clean_token.h5')  
    ######predicts_body_V2 = model_body_V2.predict(x_test_body_V2, batch_size=8)
    predicts_body_V2 = model_body_V2.predict(x_test_body_V2, batch_size=8)

CPU times: user 1d 4h 12min 19s, sys: 53min, total: 1d 5h 5min 20s
Wall time: 4h 28min 9s


> La prédiction du Body est aussi longue avec le plongement : 4h30 

##### Title

In [20]:
%%time
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model_title.load_weights('./MODELS/elmo-model-title.h5')  
    predicts_title = model_title.predict(x_test_title, batch_size=8)

CPU times: user 54min 32s, sys: 1min 53s, total: 56min 25s
Wall time: 8min 38s


##### On fait une sauvegarde intermédiaire des données pour pouvoir les analyser après coup

In [21]:
pd.DataFrame(predicts_title).to_csv('res_body_clean2/V1_predict_title.csv', index=True)
pd.DataFrame(predicts_body_V2).to_csv('res_body_clean2/V1_predict_body_clean.csv', index=True)
pd.DataFrame(y_test).to_csv('res_body_clean2/V1_y_test.csv', index=True)
#####pd.DataFrame(y_test_global).to_csv('res_body_clean2/V1_y_test_global.csv', index=True)

In [22]:
## On décode
m_predicts_title = decode(le, predicts_title)
m_predicts_body_V2 = decode(le, predicts_body_V2)
m_y_test = decode(le, y_test)

pd.DataFrame(m_predicts_title).to_csv('res_body_clean2/V2_predict_title.csv', index=True)
pd.DataFrame(m_predicts_body_V2).to_csv('res_body_clean2/V2_predict_body_clean.csv', index=True)
pd.DataFrame(m_y_test).to_csv('res_body_clean2/V2_y_test.csv', index=True)

/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Datasets décodés

In [23]:
mm_predicts_title = pd.DataFrame(np.where(m_predicts_title=='non', 0, 1), columns=['python'])
mm_predicts_body_V2 = pd.DataFrame(np.where(m_predicts_body_V2=='non', 0, 1), columns=['python'])
mm_y_test = pd.DataFrame(np.where(m_y_test=='non', 0, 1), columns=['python'])

mm_predicts_title.to_csv('res_body_clean2/V3_predict_title.csv', index=True)
mm_predicts_body_V2.to_csv('res_body_clean2/V3_predict_body_clean.csv', index=True)
mm_y_test.to_csv('res_body_clean2/V3_y_test.csv', index=True)

#### Fabrication du Dataset OR entre les prédiction du Titre et du Body

In [26]:
mm_predicts_title_OR_body_V2 = pd.DataFrame(
    np.where(np.logical_or(mm_predicts_title, mm_predicts_body_V2), 1, 0),
    columns=mm_predicts_title.columns, index=mm_predicts_title.index)

### Mesure de la performance de la version ELMO

In [27]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

l_tags = ['python']
m_ELMO2_body = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])
m_ELMO2_title = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])
m_ELMO2_title_OR_body = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])

for i_y_pred, col in enumerate(l_tags):
    # Title
    m_ELMO2_title.loc[col, 'f1_score'] = f1_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title[col].to_dense().ravel())
    
    m_ELMO2_title.loc[col, 'recall'] = recall_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title[col].to_dense().ravel())
    
    m_ELMO2_title.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title[col].to_dense().ravel())
    
    # Body
    m_ELMO2_body.loc[col, 'f1_score'] = f1_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_body_V2[col].to_dense().ravel())
    
    m_ELMO2_body.loc[col, 'recall'] = recall_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_body_V2[col].to_dense().ravel())
    
    m_ELMO2_body.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_body_V2[col].to_dense().ravel())
    
    # Title OR Body
    m_ELMO2_title_OR_body.loc[col, 'f1_score'] = f1_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title_OR_body_V2[col].to_dense().ravel())
    
    m_ELMO2_title_OR_body.loc[col, 'recall'] = recall_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title_OR_body_V2[col].to_dense().ravel())
    
    m_ELMO2_title_OR_body.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=mm_y_test[col].ravel(), 
        y_pred=mm_predicts_title_OR_body_V2[col].to_dense().ravel())
    
Total_python = pd.concat([m_ELMO2_title, m_ELMO2_body, m_ELMO2_title_OR_body], )
Total_python.index=['python_title', 'python_body', 'python_OR']
Total_python

,f1_score,recall,accuracy_score
python_title,0.395574,0.256043,0.9126
python_body,0.118812,0.0644584,0.8932
python_OR,0.432181,0.290958,0.9146


> Les résultats sont très mauvais car très faibles notamment au niveau du body

> Les résultats (accuracy) sont cependant très proches des scores d'entrainement des réseaux : 
> - Pour le title 0.9126 en test vs 0.9124 à l'entrainement
> - Pour le body 0.8932 en test vs 0.8949 à l'entrainement